In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [31]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [32]:
cwd = os.getcwd()
print(cwd)
cwd_parent = os.path.abspath(os.path.join(cwd, os.pardir))
print(cwd_parent)

sys.path.append(cwd_parent)

/home/ubuntu/mnt/graphnn/notebooks
/home/ubuntu/mnt/graphnn


In [33]:
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.chemfeatures import *
from deepadr.train_functions_flat import *
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN

In [34]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [35]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [36]:
n_gpu = torch.cuda.device_count()
n_gpu

1

In [37]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

In [38]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 2.0.0+cu117
CUDA: 11.7
3.10.9 (main, Jan 11 2023, 15:21:40) [GCC 11.2.0]


### Preparing dataset 

In [39]:
# options: 
# 'total_thresh' + 4,3,2
# 'loewe_thresh', 'hsa_thresh', 'bliss_thresh', 'zip_thresh' + 1

score = 'zip_thresh'
score_val = 1

In [40]:
DSdataset_name = f'DrugComb_{score}_{score_val}'

data_fname = 'data_v1' # v2 for baseline models, v3 for additive samples

In [41]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1


### Hyper Parameters

In [42]:
targetdata_dir_exp

'/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments'

In [43]:
#time_stamp = "2022-09-14_17-49-43"
time_stamp= '2024-01-17'


In [53]:
import glob
exp_dirs = glob.glob(targetdata_dir_exp+"/Cell_Line_ID_fold_*_"+time_stamp)
len(exp_dirs)
#/Cell_Line_ID_fold
print(exp_dirs)

['/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments/Cell_Line_ID_fold_1_2024-01-17', '/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments/Cell_Line_ID_fold_4_2024-01-17', '/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments/Cell_Line_ID_fold_0_2024-01-17', '/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments/Cell_Line_ID_fold_2_2024-01-17', '/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments/Cell_Line_ID_fold_3_2024-01-17']


In [54]:
folds = []
for edir in exp_dirs:
    fold = edir.split("/")[-1].split('_')[4]
    print(fold)
    df_curves = pd.read_csv(edir + "/curves.csv")
    df_curves["fold"] = [fold] * len(df_curves)
    folds.append(df_curves)

1
4
0
2
3


In [55]:
df_folds = pd.concat(folds)
df_folds

,epoch,train_aupr,valid_aupr,test_aupr,train_auc,valid_auc,test_auc,fold
0,0,0.786644,0.696719,0.000000,0.726307,0.664683,0.000000,1
1,1,0.811408,0.706871,0.000000,0.762315,0.664364,0.000000,1
2,2,0.841200,0.707988,0.000000,0.798981,0.666309,0.000000,1
3,3,0.862725,0.712564,0.000000,0.821482,0.668397,0.000000,1
4,4,0.881322,0.718668,0.000000,0.845849,0.672080,0.000000,1
...,...,...,...,...,...,...,...,...
95,95,0.999273,0.754358,0.000000,0.998994,0.680223,0.000000,3
96,96,0.999700,0.743609,0.000000,0.999517,0.678288,0.000000,3
97,97,0.999827,0.774389,0.000000,0.999739,0.697236,0.000000,3
98,98,0.999928,0.758448,0.000000,0.999892,0.684044,0.000000,3


In [56]:
df_folds_epoch = df_folds.groupby("epoch")[["test_aupr", "test_auc"]].mean()
df_folds_epoch["Fscore"] = df_folds_epoch.apply(lambda x: (2*x[0]*x[1])/(x[0]+x[1]+1e-7), axis=1) # harmonic mean of AUC, AUPR
df_folds_epoch

,test_aupr,test_auc,Fscore
epoch,,,
0,0.00000,0.000000,0.000000
1,0.00000,0.000000,0.000000
2,0.00000,0.000000,0.000000
3,0.00000,0.000000,0.000000
4,0.00000,0.000000,0.000000
...,...,...,...
95,0.00000,0.000000,0.000000
96,0.00000,0.000000,0.000000
97,0.00000,0.000000,0.000000


In [57]:
best_epoch = df_folds_epoch.Fscore.idxmax()
df_folds_epoch.loc[best_epoch]

test_aupr    0.764130
test_auc     0.713234
Fscore       0.737805
Name: 99, dtype: float64

### Predictions (Best Epoch)

In [58]:
print("best epoch:", best_epoch)

l_pred = []

for edir in exp_dirs:
    predictions = pd.read_csv(edir + f"/predictions/epoch_{best_epoch}_predictions_test.csv")
    l_pred.append(predictions)

best epoch: 99


In [59]:
df_pred = pd.concat(l_pred, axis=0).astype({"id":int}).set_index("id")
df_pred

,true_class,pred_class,prob_score_class0,prob_score_class1
id,,,,
47,0,0,0.922363,0.077637
48,0,0,0.830951,0.169049
49,1,1,0.029431,0.970569
50,0,1,0.077161,0.922839
51,1,1,0.051173,0.948827
...,...,...,...,...
88989,1,1,0.366748,0.633252
88990,1,1,0.190404,0.809596
88991,1,1,0.446142,0.553858


### DATA

In [47]:
data_pairs = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'data_pairs.pkl'))
data_pairs

,Drug1_ID,Drug2_ID,Cell_Line_ID,Cosmic_ID,Drug1,Drug2,Y
0,dacarbazine,717906-29-1,A-673,684052,CN(C)N=NC1=C(NC=N1)C(=O)N,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
1,CHEMBL3348974,717906-29-1,A-673,684052,CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)N...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1
2,Nesbuvir,717906-29-1,A-673,684052,CNC(=O)C1=C(OC2=CC(=C(C=C21)C3CC3)N(CCO)S(=O)(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0
3,BMS-599626,717906-29-1,A-673,684052,CC1=C2C(=NC=NN2C=C1NC(=O)OCC3COCCN3)NC4=CC5=C(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1
4,WZ8040,717906-29-1,A-673,684052,CN1CCN(CC1)C2=CC=C(C=C2)NC3=NC=C(C(=N3)SC4=CC=...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1
...,...,...,...,...,...,...,...
89042,Vorinostat,KU-60019,VM-CUB-1,909780,C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO,CC1CN(CC(O1)C)CC(=O)NC2=CC3=C(C=C2)SC4=C(C=CC=...,1
89043,AZD5582,AZD4320,VM-CUB-1,909780,CC(C(=O)NC(C1CCCCC1)C(=O)N2CCCC2C(=O)NC3C(CC4=...,CN(CCC(CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O)(=O)N...,0
89044,957054-30-7,Sapitinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CNC(=O)CN1CCC(CC1)OC2=C(C=C3C(=C2)C(=NC=N3)NC4...,1
89045,957054-30-7,Selumetinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,1


In [60]:
df_attn = None
if (df_attn is not None):
    df_all = pd.concat([data_pairs, df_attn, df_pred], axis=1)
else: 
    df_all = pd.concat([data_pairs, df_pred], axis=1)
    
df_all=df_all.astype({"true_class":int})
df_all

,Drug1_ID,Drug2_ID,Cell_Line_ID,Cosmic_ID,Drug1,Drug2,Y,true_class,pred_class,prob_score_class0,prob_score_class1
0,dacarbazine,717906-29-1,A-673,684052,CN(C)N=NC1=C(NC=N1)C(=O)N,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0,0,1,0.046764,0.953236
1,CHEMBL3348974,717906-29-1,A-673,684052,CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)N...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1,1,1,0.045205,0.954795
2,Nesbuvir,717906-29-1,A-673,684052,CNC(=O)C1=C(OC2=CC(=C(C=C21)C3CC3)N(CCO)S(=O)(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,0,0,1,0.161351,0.838649
3,BMS-599626,717906-29-1,A-673,684052,CC1=C2C(=NC=NN2C=C1NC(=O)OCC3COCCN3)NC4=CC5=C(...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1,1,1,0.106453,0.893547
4,WZ8040,717906-29-1,A-673,684052,CN1CCN(CC1)C2=CC=C(C=C2)NC3=NC=C(C(=N3)SC4=CC=...,CN(C1=CC=CC=C1CNC2=NC(=NC=C2C(F)(F)F)NC3=CC4=C...,1,1,1,0.026652,0.973348
...,...,...,...,...,...,...,...,...,...,...,...
89042,Vorinostat,KU-60019,VM-CUB-1,909780,C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO,CC1CN(CC(O1)C)CC(=O)NC2=CC3=C(C=C2)SC4=C(C=CC=...,1,1,1,0.092771,0.907229
89043,AZD5582,AZD4320,VM-CUB-1,909780,CC(C(=O)NC(C1CCCCC1)C(=O)N2CCCC2C(=O)NC3C(CC4=...,CN(CCC(CSC1=CC=CC=C1)NC2=C(C=C(C=C2)S(=O)(=O)N...,0,0,1,0.124291,0.875709
89044,957054-30-7,Sapitinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CNC(=O)CN1CCC(CC1)OC2=C(C=C3C(=C2)C(=NC=N3)NC4...,1,1,1,0.054472,0.945528
89045,957054-30-7,Selumetinib,VM-CUB-1,909780,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,1,1,1,0.036390,0.963610


In [61]:
targetdata_dir_exp

'/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments'

In [62]:
dir_fold_all = create_directory(os.path.join(targetdata_dir_exp, "all_fold_" + time_stamp))
print(dir_fold_all)

/home/ubuntu/mnt/graphnn/data/processed/DrugComb_zip_thresh_1/data_v1/experiments/all_fold_2024-01-17


In [63]:
ReaderWriter.dump_data(df_all, os.path.join(dir_fold_all, f'drugcomb_attn_predictions_{score}_{score_val}.pkl'))

In [64]:
score

'zip_thresh'

In [65]:
score_val

1